# Basic Flopy

In this notebook, we will blast through basic flopy usage.  There is a lot to cover in flopy, so this will just hit the highlights

In [ ]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import flopy
import pyemu

Let's jump right in and load an existing model (one we will use for the rest of the week). We will use the `model_ws` (model workspace) to access these files in a different directory:

In [ ]:
model_ws = os.path.join("..","base_model_files")
os.listdir(model_ws)


Do these file types look familiar???

In [ ]:
m = flopy.modflow.Modflow.load("freyberg.nam",model_ws=model_ws,verbose=True)

In [ ]:
m

individual packages are accessible through their 3-letter extension (except for the basic package, which is `bas6` for some reason :facepalm: )

In [ ]:
m.dis

In each package, the array-type data are housed in special array-handling classes:

In [ ]:
m.dis.top

In [ ]:
m.rch.rech

In [ ]:
m.bas6.strt

List-type data also have special handlers - the list type data is always stored in the `stress_period_data` attribute:

In [ ]:
m.drn.stress_period_data

And of course, `SFR` has to be difficult

In [ ]:
pd.DataFrame.from_records(m.sfr.reach_data)

In [ ]:
pd.DataFrame.from_records(m.sfr.segment_data[0])

## Exporting to shapefiles

You can export the entire model, individual packages or individual attributes:

In [ ]:
m.export("model.shp")
m.dis.export("dis.shp")
m.dis.top.export("top.shp")
m.drn.stress_period_data.export("drn.shp")

How does flopy know where the model is in space? The spatial reference:

In [ ]:
m.sr

# changing `model_ws` and writing

In [ ]:
m.change_model_ws("flopy_temp",reset_external=True)
m.write_input()


In [ ]:
os.listdir(m.model_ws)

# Run the model

A little trickery is needed here: we need to get the right MODFLOW binary into this new `model_ws`.  What should we do? 

### DIY: use `shutil` and `os` to copy the `mfnwt` binary into the new `model_ws`

In [ ]:
os.listdir(bin_dir)
# your code here:

In [ ]:
pyemu.os_utils.run("mfnwt freyberg.nam",cwd=m.model_ws)

# Post-processing

Flopy has lots of support to help use deal with the terrible MDOFLOW file formats

In [ ]:
mflist = flopy.utils.MfListBudget(os.path.join(m.model_ws,m.name+".list"))

In [ ]:
inc_df,cum_df = mflist.get_dataframes()
inc_df

Where did those dates come from?  Check this out:

In [ ]:
m.start_datetime

### DIY: plot the net increment water budget as a bar chart but change the starting datetime to 5 Nov 1955:

In [ ]:
# your code here

We can also load the binary head save file

In [ ]:
hds = flopy.utils.HeadFile(os.path.join(m.model_ws,m.name+".hds"))
hds

In [ ]:
hds.get_times()

In [ ]:
data = hds.get_data() #by defauly, get_data() returns the last entry
data.shape

In [ ]:
hds.plot()

That looks kinda shitty...but we can do better! if you pass the model instance the the `HeadFile` constructor, the plot routines will use the ibound to mask...

### DIY: re-instantiate the `HeadFile` object but pass it the model instance and call plot again:

The `HeadFile` class also support writing to shapefiles:

hds.to_shapefile("hds.shp")

### DIY: change the `model_ws` again (and get the binary into the new `model_ws`).  Then change everyone's fav hydrogeoloical obsession `hk` in each layer by a random factor, run the model and compare the water budget and heads to the base case we ran through above.

# Creating a new model and adding packages

Now we will go through the super painful process of creating a super simple model

In [ ]:
m = flopy.modflow.Modflow("newmodel",model_ws="newmodel",version="mfnwt",exe_name="mfnwt")

In [ ]:
dis = flopy.modflow.ModflowDis(m,nrow=1,ncol=10,nlay=1,nper=2,top=10,botm=0,steady=[True,False],perlen=[1.0,10])

In [ ]:
ibound = np.ones((m.nlay,m.nrow,m.ncol))
ibound[:,:,[0,-1]] = -1
strt = np.ones_like(ibound)
strt[:,:,-1] = 10
bas = flopy.modflow.ModflowBas(m)
m.bas6.ibound = ibound
m.bas6.strt=strt

In [ ]:
upw = flopy.modflow.ModflowUpw(m,hk=10)

In [ ]:
nwt = flopy.modflow.ModflowNwt(m)

In [ ]:
oc = flopy.modflow.ModflowOc(m)

In [ ]:
# to add a boundary condition, we need to generate a dict of stress period key and list-type data values
# for the wel package, the list type data need to have k, i, j, and flux
spd = {0:[0,0,4,-10],1:[0,0,4,-20]}

In [ ]:
wel = flopy.modflow.ModflowWel(m,stress_period_data=spd)

In [ ]:
m.write_input()

In [ ]:
shutil.copy2(os.path.join("bin","mac","mfnwt"),os.path.join(m.model_ws,"mfnwt"))

In [ ]:
pyemu.os_utils.run("mfnwt newmodel.nam",cwd=m.model_ws)

In [ ]:
hds = flopy.utils.HeadFile(os.path.join(m.model_ws,m.name+".hds"))
hds.plot()

In [ ]:
mflist = flopy.utils.MfListBudget(os.path.join(m.model_ws,m.name+".list"))
inc,cum = mflist.get_dataframes(diff=True)
inc.plot(kind='bar',figsize=(10,10))

# DIY: create a 3 layer, 11 row, 11 col model with 365 daily stress periods (first stress period steady-state).  Put constant heads in layer 1 on the left and right with a gradient across the model and a single pumping well in the center of layer 3 and drive the pumping well with random values for each day. Run this model and post-process as above.